# Diversity Statements Scraper

In [2]:
# Data manipulation libraries
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
# Common webscraping libaries
from bs4 import BeautifulSoup as bs
import requests

tqdm.pandas()

### Get list of universities

In [5]:
# input_path = "../data/input/pilot_websites.csv"
input_path = "../data/input/PART 1_Scraping Diversity Statement Website Links.csv"
uni_list = pd.read_csv(input_path)
# uni_list = uni_list.melt(id_vars=['University'], 
#               value_vars=[x for x in uni_list.columns if "WebPage" in x or "Admissions" in x], 
#               var_name='url_type', value_name='url')
# uni_list["url_type"],uni_list["url_rank"] = uni_list["url_type"].str[:-3], uni_list["url_type"].str[-3:]
# uni_list = uni_list[["University","url_type","url_rank","url"]].dropna(subset=["url"])
# print(uni_list.shape)
# uni_list = uni_list.drop_duplicates(subset="url")
print(uni_list.shape)
uni_list

(37, 2)


,School Name,UniversityLink1
0,Princeton University,https://inclusive.princeton.edu/about/our-comm...
1,Harvard University,https://college.harvard.edu/life-at-harvard/di...
2,Massachusetts Institute of Technology,https://diversity.mit.edu/about/about-institut...
3,Yale University,https://belong.yale.edu/we-are-yale-university
4,Stanford University,https://diversityworks.stanford.edu/
5,University of Chicago,https://diversityandinclusion.uchicago.edu/
6,University of Pennsylvania,https://diversity.upenn.edu/diversity-at-penn
7,California Institute of Technology,https://breakthrough.caltech.edu/ways-to-give/...
8,Johns Hopkins University,https://diversity.jhu.edu/our-commitment/
9,Northwestern University,https://www.northwestern.edu/diversity/about/m...


In [8]:
from utils import scrape_webpage
uni_list['text'] = uni_list.progress_apply(lambda x : scrape_webpage(x["UniversityLink1"],
                                                            x["School Name"],
#                                                             x["url_type"],
                                                            out_folder="output",
                                                            save_images=False),axis=1)
uni_list['failed_scrape'] = uni_list['text'].str.len() < 100
# uni_list[['University','url_type','url_rank','url','failed_scrape','text']]
uni_list.to_csv(f"../data/output/pilot_websites.csv")

  0%|          | 0/37 [00:00<?, ?it/s]